In [ ]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
import numpy as np
%matplotlib inline
category = "Video_Games"

In [ ]:
df_multiple = getDF('../../data/answers_multiple/QA_'+category+'.json.gz')

In [ ]:
df = getDF('../../data/reviews_small/QA_'+category+'.json.gz')

In [ ]:
print "Single Answers DF Columns", list(df)
print "Multiple Answers DF Columns", list(df_multiple)

In [ ]:
print "questions dictionary keys in multiple DF", df_multiple['questions'][0][0].keys()

In [ ]:
def pid_question_answers(row):
    questions = row['questions']
    total_questions = len(questions)
    num_answers_list = [len(question['answers']) for question in questions]
    total_answers = sum(num_answers_list)
    return (row['asin'], total_questions, total_answers)

In [ ]:
df_pid_metrics = df_multiple[['asin', 'questions']].apply(pid_question_answers, axis=1)
df_pid_metrics = df_pid_metrics.apply(pd.Series)
df_pid_metrics.columns=['asin', '#Questions','#Answers']
print df_pid_metrics

In [ ]:
df_pid_metrics['#Questions'].value_counts().sort_index().plot(kind='bar')

In [ ]:
def qid_answers(questions):
    num_answers_list = [len(question['answers']) for question in questions]
    return num_answers_list

In [ ]:
df_qid_metrics = df_multiple['questions'].apply(qid_answers)
l = df_qid_metrics.tolist()
flat_list = [item for sublist in l for item in sublist]
df_qid_metrics = pd.DataFrame(np.array(flat_list), columns=['#Answers'])

In [ ]:
df_qid_metrics

In [ ]:
print sum(df_qid_metrics['#Answers'].tolist())
print df_qid_metrics.describe()

In [ ]:
df_qid_metrics['#Answers'].value_counts().sort_index().plot(kind='bar')

In [ ]:
def aid_helpful(questions):
    helpful_list = [tuple(answer['helpful']) for question in questions for answer in question['answers']]
    return helpful_list

In [ ]:
df_aid_metrics = df_multiple['questions'].apply(aid_helpful)
l = df_aid_metrics.tolist()
flat_list = [item for sublist in l for item in sublist]
df_aid_metrics = pd.DataFrame(np.array(flat_list), columns=['yes', 'no'])

In [ ]:
# print df_aid_metrics
df_aid_metrics.describe()

In [ ]:
df_aid_metrics['yes'].value_counts().sort_index().plot(kind='bar')

QA data with asin and List of Questions. Each Question has List of Answers

In [ ]:
df_multiple['questions']

In [ ]:
reviews_df = getDF('../../data/reviews_'+category+'_5.json.gz')

In [ ]:
reviews_df

In [ ]:
def convertRowToJson(row):
    json = {}
    json['reviewText'] = row['reviewText']
    json['helpful'] = row['helpful']
    json['summary'] = row['summary']
    return json

In [ ]:
reviews_df['reviews'] = reviews_df[['reviewText', 'helpful', 'summary']].apply(convertRowToJson, axis=1)

In [ ]:
reviews_df['reviews'][0]

In [ ]:
def convert(columns):
    newlist = sorted(columns['reviews'].tolist(), key=lambda k: k['helpful'][0], reverse=True) 
    return newlist[:5]

In [ ]:
reviews_filtered_df = reviews_df.groupby('asin').apply(convert).reset_index()
reviews_filtered_df.columns = ['asin', 'reviews']

In [ ]:
qa_reviews_df = pd.merge(df_multiple, reviews_filtered_df, on=['asin', 'asin'])

In [ ]:
qa_reviews_df

In [ ]:
import nltk
tokens = nltk.word_tokenize("Hello Hi")
tokens

In [ ]:
def len_reviews(reviews):
    count = [len(nltk.word_tokenize(review['reviewText'])) for review in reviews]
    return count

In [ ]:
qa_reviews_df['reviews'].apply(len_reviews)

In [ ]:
def questionTypes(questions):
    questionTypes = [question['questionType'] for question in questions]
    return questionTypes

In [ ]:
df_question_type_metrics = qa_reviews_df['questions'].apply(questionTypes)
l = df_question_type_metrics.tolist()
flat_list = [item for sublist in l for item in sublist]
df_question_type_metrics = pd.DataFrame(np.array(flat_list))

In [ ]:
df_question_type_metrics[0].value_counts().sort_index().plot(kind='bar')